# Extraction of Frequency Bands and Asymetry Data


In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
%matplotlib inline

### Create marker list
- Here the idea is to seperae the EEG data recorded for one participant into t(s) epochs (example t = 15s). 
- This yeilds more sampltes to train, validate and test our models

In [2]:
#Create marker list
def get_markers(epoch_length=15):
    sampling_rate = 125 # 125 Hz
    min_frac = (epoch_length/60.0)
    step_minutes = np.arange(0,6,min_frac) # Start - 0 min, total_recording_time - 6 min, step = 15s(i.e. 0.25 min)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers

epoch_length = 15 # This is 15 seconds, can be reduced to 10s if we need more data
markers = get_markers(epoch_length=epoch_length)
len(markers)

step_minutes=array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75, 5.  , 5.25,
       5.5 , 5.75])


24

### Convert to mne.io.RawArray

In [3]:
#Convert a Pandas df to mne.io.RawArray
def dataframe_to_raw(dataframe, sfreq):
    ch_names = list(dataframe.columns)
    ch_types = ['eeg'] * (len(dataframe.columns))
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    dataframe = dataframe.T  # mne looks at the tranpose() format
    dataframe *= 1e-6  # convert from uVolts to Volts (mne assumes Volts data)

    info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq, verbose=False)

    raw = mne.io.RawArray(dataframe, info,verbose=False)
    raw.set_montage(ten_twenty_montage)
    
    return raw

### Read labels CSV

In [4]:
# Read the csv file with lables and put id, lable into a dict.
def read_labels(file_path):
    label_df = pd.read_csv(file_path)
    data = label_df[['id', 'label']]
    data = data.to_numpy()
    dict_label = dict()

    for i in range(len(data)):
        dict_label[data[i][0]] = data[i][1]
    return dict_label

label_file_path = '../data/labels/PSS10-Sheet1.csv'
dict_label = read_labels(label_file_path)    
print(dict_label)

{1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 1}


### Export PSD featurs from all ch_16_csv files

In [5]:
# Reference1 : https://github.com/akraradets/EEG_ChronicStress/blob/main/2-ChronicStress/3-export-feature.ipynb
# Reference2 : https://github.com/chaklam-silpasuwanchai/Python-for-EEG/blob/main/03%20-%20Asymmetry%20%2B%20DEAP.ipynb

def export_features_to_csv(in_dir_path, out_dir_path, file_name):

    #dir_path = '../data/ch_16_csv'
    #f_name = 'conv_exp01.csv'
    f_path = in_dir_path + '/' + file_name
    # Read csv
    in_df = pd.read_csv(f_path)

    f_idx = file_name[3:]
    f_idx = f_idx[:-4]
    f_id = int(f_idx)
    label = dict_label[f_id]
    print(f_id, label)

    # df to mne.io.RawArray
    raw = dataframe_to_raw(in_df, 125)
    
    #Artifact removal
    raw.notch_filter(50) # 50 Hz from electrical network
    #Slow drift and high freq. removal
    raw.filter(1, 43)
    
    #Epochs
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=8, baseline=None, verbose=False) #Our recording time is 6min thus as tmax 8min is used
    spectrum = epochs.compute_psd(method="welch", n_fft=125)  # Get Spectrum object from epochs

    #The shape of PSDs is (n_epochs,n_channels,n_freqs) and FREQs is (n_freqs)
    psds, freqs = spectrum.get_data(return_freqs=True) 
    # psds.shape, freqs.shape, (24, 16, 63) (63,)
    psds = 10 * np.log10(psds)

    band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta', 'Relative_gamma'])
    mean_band_names = np.array(['mean_Delta', 'mean_Theta', 'mean_Alpha', 'mean_Beta', 
                                'mean_Gamma', 'mean_Slow', 'mean_Low_beta', 'mean_Relative_gamma'])
    #filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]

    #Take the mean of all channels
    mean_psds = psds.mean(axis=1)
    #mean_psds.shape, (24, 63)

    #Band mean extraction
    delta = mean_psds[:, 1:3]
    theta = mean_psds[:, 4:7]
    alpha = mean_psds[:, 8:12]
    beta  = mean_psds[:, 13:30]
    gamma = mean_psds[:, 30:43]
    slow = mean_psds[:, 4:13]
    low_beta = mean_psds[:, 13:17]

    #For each band take the mean of the power
    delta = delta.mean(axis=1).reshape(-1, 1)
    theta = theta.mean(axis=1).reshape(-1, 1)
    alpha = alpha.mean(axis=1).reshape(-1, 1)
    beta  = beta.mean(axis=1).reshape(-1, 1)
    gamma = gamma.mean(axis=1).reshape(-1, 1)
    slow = slow.mean(axis=1).reshape(-1, 1)
    low_beta = low_beta.mean(axis=1).reshape(-1, 1)
    relative_gamma = slow/gamma

    #Stack them together
    _temp = [delta, theta, alpha, beta, gamma, slow, low_beta, relative_gamma]
    all_bands = np.hstack(_temp)
    #all_bands.shape , (24, 7)

    out_df = pd.DataFrame(all_bands, columns = mean_band_names.tolist())

    # Jo's channels ['Fp1','Fp2','F3','F4','F7','F8','C3','C4','T3','T4','T5','T6','P3','P4','O1','O2']
    # Our channels['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']

    # Band extraction
    channel_list = ['Fp1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T3', 'C3', 'C4', 'T4', 'T5', 'P3', 'P4', 'T6', 'O1', 'O2']

    for i in range(len(channel_list)):
        new_band_names = []
        for band_name in band_names:
            new_name = channel_list[i] + '_' + band_name
            new_band_names.append(new_name)

        delta = psds[:, i, 1:3]
        theta = psds[:, i, 4:7]
        alpha = psds[:, i, 8:12]
        beta  = psds[:, i, 13:30]
        gamma = psds[:, i, 30:43]
        slow = psds[:, i, 4:13]
        low_beta = psds[:, i, 13:17]

        #For each band take the mean of the power
        delta = delta.mean(axis=1).reshape(-1, 1)
        theta = theta.mean(axis=1).reshape(-1, 1)
        alpha = alpha.mean(axis=1).reshape(-1, 1)
        beta  = beta.mean(axis=1).reshape(-1, 1)
        gamma = gamma.mean(axis=1).reshape(-1, 1)
        slow = slow.mean(axis=1).reshape(-1, 1)
        low_beta = low_beta.mean(axis=1).reshape(-1, 1)
        relative_gamma = slow/gamma

        #Stack them together
        _temp = [delta, theta, alpha, beta, gamma, slow, low_beta, relative_gamma]
        all_bands = np.hstack(_temp)
        temp_df = pd.DataFrame(all_bands, columns = new_band_names)
        out_df = pd.concat([out_df, temp_df], axis=1, join='inner')
    
    # alpha_frontal asymetry
    a_f4 = psds[:, 4, 8:12]
    a_f4 = a_f4.mean(axis=1).reshape(-1, 1)
    a_f3 = psds[:, 3, 8:12]
    a_f3 = a_f3.mean(axis=1).reshape(-1, 1)

    alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
    out_df['alpha_frontal'] = alpha_frontal[0]

    # alpha_temporal asymetry
    a_t4 = psds[:, 9, 8:12]
    a_t4 = a_t4.mean(axis=1).reshape(-1, 1)
    a_t3 = psds[:, 6, 8:12]
    a_t3 = a_t3.mean(axis=1).reshape(-1, 1)

    alpha_temporal = ((a_t4 - a_t3) / (a_t4 + a_t3)).reshape(1,-1)
    out_df['alpha_temporal'] = alpha_temporal[0]

    # alpha_asymmetry
    alpha_asymmetry = alpha_frontal[0] + alpha_temporal[0]
    out_df['alpha_asymmetry'] = alpha_asymmetry

    # beta_frontal asymetry
    b_f4 = psds[:, 4, 13:30] # F4 index is 4
    b_f4 = b_f4.mean(axis=1).reshape(-1, 1)
    b_f3 = psds[:, 3, 13:30] # F3 index is 3
    b_f3 = b_f3.mean(axis=1).reshape(-1, 1)

    beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
    out_df['beta_frontal'] = beta_frontal[0]

    # beta_temporal asymetry
    b_t4 = psds[:, 9, 13:30] # T4 index is 9
    b_t4 = b_t4.mean(axis=1).reshape(-1, 1)
    b_t3 = psds[:, 6, 13:30] # T3 index is 6
    b_t3 = b_t3.mean(axis=1).reshape(-1, 1)

    beta_temporal = ((b_t4 - b_t3) / (b_t4 + b_t3)).reshape(1,-1)
    out_df['beta_temporal'] = beta_temporal[0]

    # beta_asymmetry
    beat_asymmetry = beta_frontal[0] + beta_temporal[0]
    out_df['beat_asymmetry'] = beat_asymmetry

    out_df['participant_id'] = f_id
    out_df['label'] = label

    out_file_path = out_dir_path + '/' + file_name
    out_df.to_csv(out_file_path,index=False)


In [6]:
in_dir_path = '../data/ch_16_csv'
out_dir_path = '../data/psd_filtered_features_csv'
file_names = ['exp01.csv', 'exp02.csv', 'exp03.csv', 'exp04.csv', 'exp05.csv', 'exp06.csv', 
              'exp07.csv', 'exp08.csv', 'exp09.csv', 'exp10.csv', 'exp11.csv', 'exp12.csv', 
              'exp13.csv', 'exp14.csv', 'exp15.csv', 'exp16.csv', 'exp17.csv']

for file_name in file_names:
    export_features_to_csv(in_dir_path, out_dir_path, file_name)

1 1
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwidth: 10.75 Hz (-6 dB cutoff frequency: 48.38 Hz)
- Filter length: 413 samples (3.304 sec)

Using data from preloaded Raw for 24 events and 1001 original time points ...
0 bad epochs dropped
Effective window size : 1.000 (s)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


2 1
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwidt

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


16 1
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwid

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


17 1
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 825 samples (6.600 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 43 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 43.00 Hz
- Upper transition bandwid

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.0s finished


In [7]:
# Test one PSD feature CSV file
out_file_path = '../data/psd_filtered_features_csv/exp10.csv'
df = pd.read_csv(out_file_path)
df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-104.408743,-116.002491,-116.245420,-122.190004,-125.852903,-115.804305,-118.668658,0.920156,-104.844748,-115.348004,...,-116.219412,0.927389,0.000327,-0.013005,-0.012678,0.001616,0.003406,0.005022,10,0
1,-105.031525,-116.379362,-117.155926,-120.757879,-125.310736,-116.615388,-117.192701,0.930610,-105.955624,-115.443576,...,-116.408793,0.937435,-0.000215,-0.016083,-0.016298,-0.002578,-0.001698,-0.004276,10,0
2,-107.393623,-115.823653,-115.647227,-121.342509,-125.310320,-115.691466,-118.316908,0.923240,-109.284696,-115.601348,...,-115.352877,0.932703,0.000453,-0.006299,-0.005845,-0.001092,-0.002972,-0.004064,10,0
3,-108.985147,-115.772532,-116.646741,-121.736901,-125.285641,-115.851506,-119.082744,0.924699,-109.244951,-116.191879,...,-115.698693,0.938104,0.001348,-0.000573,0.000775,-0.002004,0.003243,0.001239,10,0
4,-104.931515,-115.248261,-116.199350,-122.291697,-125.620539,-115.987008,-119.518244,0.923312,-104.721302,-113.416372,...,-116.860585,0.939116,-0.004569,-0.022048,-0.026617,-0.002335,-0.000150,-0.002485,10,0


### Export all PSD feature CSV files to one CSV file

In [8]:
dir_path = '../data/psd_filtered_features_csv'
file_names_from_second = ['exp02.csv', 'exp03.csv', 'exp04.csv', 'exp05.csv', 'exp06.csv', 
              'exp07.csv', 'exp08.csv', 'exp09.csv', 'exp10.csv', 'exp11.csv', 'exp12.csv', 
              'exp13.csv', 'exp14.csv', 'exp15.csv', 'exp16.csv', 'exp17.csv']
file_path = '../data/psd_filtered_features_csv/exp01.csv'
all_df = pd.read_csv(file_path)
all_df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-114.341825,-121.488572,-123.828235,-131.472169,-135.139612,-123.128543,-128.672829,0.911121,-108.585469,-117.433845,...,-127.824557,0.900983,-0.002424,-0.001148,-0.003572,-0.002992,0.010864,0.007871,1,1
1,-114.839073,-121.304136,-123.781423,-130.799737,-134.596977,-123.236447,-128.949312,0.915596,-110.156485,-120.517707,...,-130.412991,0.916433,-0.005414,-0.008728,-0.014143,-0.001580,0.003520,0.001940,1,1
2,-113.316245,-120.735108,-124.310022,-130.503170,-134.414663,-123.339975,-130.710334,0.917608,-111.637451,-117.818647,...,-131.930584,0.918449,-0.003896,-0.004369,-0.008265,0.000281,0.001873,0.002154,1,1
3,-113.712446,-122.213827,-123.576121,-128.656480,-135.155641,-123.311830,-127.730350,0.912369,-109.539221,-119.066950,...,-129.857027,0.925494,-0.005195,0.010574,0.005379,-0.005731,0.003617,-0.002114,1,1
4,-114.765877,-120.000556,-123.287467,-130.629803,-135.184171,-122.322574,-129.815779,0.904859,-109.297915,-117.081706,...,-131.320020,0.901602,0.001199,-0.002400,-0.001201,-0.007199,0.008931,0.001733,1,1


In [9]:
for file_name in file_names_from_second:
    file_path = dir_path + '/' + file_name
    temp_df = pd.read_csv(file_path)
    frames = [all_df, temp_df]
    all_df = pd.concat(frames,ignore_index=True)

In [10]:
all_df.head(-5)

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-114.341825,-121.488572,-123.828235,-131.472169,-135.139612,-123.128543,-128.672829,0.911121,-108.585469,-117.433845,...,-127.824557,0.900983,-0.002424,-0.001148,-0.003572,-0.002992,0.010864,0.007871,1,1
1,-114.839073,-121.304136,-123.781423,-130.799737,-134.596977,-123.236447,-128.949312,0.915596,-110.156485,-120.517707,...,-130.412991,0.916433,-0.005414,-0.008728,-0.014143,-0.001580,0.003520,0.001940,1,1
2,-113.316245,-120.735108,-124.310022,-130.503170,-134.414663,-123.339975,-130.710334,0.917608,-111.637451,-117.818647,...,-131.930584,0.918449,-0.003896,-0.004369,-0.008265,0.000281,0.001873,0.002154,1,1
3,-113.712446,-122.213827,-123.576121,-128.656480,-135.155641,-123.311830,-127.730350,0.912369,-109.539221,-119.066950,...,-129.857027,0.925494,-0.005195,0.010574,0.005379,-0.005731,0.003617,-0.002114,1,1
4,-114.765877,-120.000556,-123.287467,-130.629803,-135.184171,-122.322574,-129.815779,0.904859,-109.297915,-117.081706,...,-131.320020,0.901602,0.001199,-0.002400,-0.001201,-0.007199,0.008931,0.001733,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,-107.455043,-111.863141,-112.702479,-114.974544,-116.724824,-112.371037,-114.331778,0.962700,-108.124007,-112.138981,...,-117.055024,0.884875,0.003017,0.001547,0.004565,-0.002119,0.000400,-0.001719,17,1
399,-109.071794,-111.925383,-113.674901,-114.619781,-116.621789,-112.988710,-115.098427,0.968847,-110.374341,-111.142424,...,-120.172430,0.924310,-0.001265,0.004639,0.003374,-0.002551,0.000480,-0.002071,17,1
400,-106.163785,-113.111650,-113.239331,-113.770027,-116.870503,-113.030485,-113.512240,0.967143,-108.662443,-114.105586,...,-116.975402,0.908349,0.000259,0.000498,0.000758,-0.002457,0.001379,-0.001079,17,1
401,-105.940171,-111.656479,-112.138943,-114.636288,-116.951515,-112.023749,-113.942169,0.957865,-101.770444,-111.716434,...,-117.990887,0.885553,-0.002615,-0.003820,-0.006434,-0.002013,-0.000365,-0.002378,17,1


In [11]:
all_df.shape

(408, 144)

In [12]:
file_path = '../data/psd_filtered_features_csv/all.csv'
all_df.to_csv(file_path,index=False)

In [13]:
# Test all.csv PSD feature CSV file
file_path = '../data/psd_filtered_features_csv/all.csv'
df = pd.read_csv(file_path)
df.head()

,mean_Delta,mean_Theta,mean_Alpha,mean_Beta,mean_Gamma,mean_Slow,mean_Low_beta,mean_Relative_gamma,Fp1_Delta,Fp1_Theta,...,O2_Low_beta,O2_Relative_gamma,alpha_frontal,alpha_temporal,alpha_asymmetry,beta_frontal,beta_temporal,beat_asymmetry,participant_id,label
0,-114.341825,-121.488572,-123.828235,-131.472169,-135.139612,-123.128543,-128.672829,0.911121,-108.585469,-117.433845,...,-127.824557,0.900983,-0.002424,-0.001148,-0.003572,-0.002992,0.010864,0.007871,1,1
1,-114.839073,-121.304136,-123.781423,-130.799737,-134.596977,-123.236447,-128.949312,0.915596,-110.156485,-120.517707,...,-130.412991,0.916433,-0.005414,-0.008728,-0.014143,-0.001580,0.003520,0.001940,1,1
2,-113.316245,-120.735108,-124.310022,-130.503170,-134.414663,-123.339975,-130.710334,0.917608,-111.637451,-117.818647,...,-131.930584,0.918449,-0.003896,-0.004369,-0.008265,0.000281,0.001873,0.002154,1,1
3,-113.712446,-122.213827,-123.576121,-128.656480,-135.155641,-123.311830,-127.730350,0.912369,-109.539221,-119.066950,...,-129.857027,0.925494,-0.005195,0.010574,0.005379,-0.005731,0.003617,-0.002114,1,1
4,-114.765877,-120.000556,-123.287467,-130.629803,-135.184171,-122.322574,-129.815779,0.904859,-109.297915,-117.081706,...,-131.320020,0.901602,0.001199,-0.002400,-0.001201,-0.007199,0.008931,0.001733,1,1


In [14]:
temp_list = list()
for i in df.head(0):
    temp_list.append(i)

In [15]:
temp_list

['mean_Delta',
 'mean_Theta',
 'mean_Alpha',
 'mean_Beta',
 'mean_Gamma',
 'mean_Slow',
 'mean_Low_beta',
 'mean_Relative_gamma',
 'Fp1_Delta',
 'Fp1_Theta',
 'Fp1_Alpha',
 'Fp1_Beta',
 'Fp1_Gamma',
 'Fp1_Slow',
 'Fp1_Low_beta',
 'Fp1_Relative_gamma',
 'Fp2_Delta',
 'Fp2_Theta',
 'Fp2_Alpha',
 'Fp2_Beta',
 'Fp2_Gamma',
 'Fp2_Slow',
 'Fp2_Low_beta',
 'Fp2_Relative_gamma',
 'F7_Delta',
 'F7_Theta',
 'F7_Alpha',
 'F7_Beta',
 'F7_Gamma',
 'F7_Slow',
 'F7_Low_beta',
 'F7_Relative_gamma',
 'F3_Delta',
 'F3_Theta',
 'F3_Alpha',
 'F3_Beta',
 'F3_Gamma',
 'F3_Slow',
 'F3_Low_beta',
 'F3_Relative_gamma',
 'F4_Delta',
 'F4_Theta',
 'F4_Alpha',
 'F4_Beta',
 'F4_Gamma',
 'F4_Slow',
 'F4_Low_beta',
 'F4_Relative_gamma',
 'F8_Delta',
 'F8_Theta',
 'F8_Alpha',
 'F8_Beta',
 'F8_Gamma',
 'F8_Slow',
 'F8_Low_beta',
 'F8_Relative_gamma',
 'T3_Delta',
 'T3_Theta',
 'T3_Alpha',
 'T3_Beta',
 'T3_Gamma',
 'T3_Slow',
 'T3_Low_beta',
 'T3_Relative_gamma',
 'C3_Delta',
 'C3_Theta',
 'C3_Alpha',
 'C3_Beta',
 'C3_

In [16]:
#cols = df.columns
#cols
#cols[60:150]

In [17]:
#temp_list = ['mean_Delta', 'mean_Theta', 'mean_Alpha', 'mean_Beta', 'mean_Gamma', 'mean_Slow', 'mean_Low_beta', 'Fp1_Delta', 'Fp1_Theta', 'Fp1_Alpha', 'Fp1_Beta', 'Fp1_Gamma', 'Fp1_Slow', 'Fp1_Low_beta', 'Fp2_Delta', 'Fp2_Theta', 'Fp2_Alpha', 'Fp2_Beta', 'Fp2_Gamma', 'Fp2_Slow', 'Fp2_Low_beta', 'F7_Delta', 'F7_Theta', 'F7_Alpha', 'F7_Beta', 'F7_Gamma', 'F7_Slow', 'F7_Low_beta', 'F3_Delta', 'F3_Theta', 'F3_Alpha', 'F3_Beta', 'F3_Gamma', 'F3_Slow', 'F3_Low_beta', 'F4_Delta', 'F4_Theta', 'F4_Alpha', 'F4_Beta', 'F4_Gamma', 'F4_Slow', 'F4_Low_beta', 'F8_Delta', 'F8_Theta', 'F8_Alpha', 'F8_Beta', 'F8_Gamma', 'F8_Slow', 'F8_Low_beta', 'T3_Delta', 'T3_Theta', 'T3_Alpha', 'T3_Beta', 'T3_Gamma', 'T3_Slow', 'T3_Low_beta', 'C3_Delta', 'C3_Theta', 'C3_Alpha', 'C3_Beta', 'C3_Gamma', 'C3_Slow', 'C3_Low_beta', 'C4_Delta', 'C4_Theta', 'C4_Alpha', 'C4_Beta', 'C4_Gamma', 'C4_Slow', 'C4_Low_beta', 'T4_Delta', 'T4_Theta', 'T4_Alpha', 'T4_Beta', 'T4_Gamma', 'T4_Slow', 'T4_Low_beta', 'T5_Delta', 'T5_Theta', 'T5_Alpha', 'T5_Beta', 'T5_Gamma', 'T5_Slow', 'T5_Low_beta', 'P3_Delta', 'P3_Theta', 'P3_Alpha', 'P3_Beta', 'P3_Gamma', 'P3_Slow', 'P3_Low_beta', 'P4_Delta', 'P4_Theta', 'P4_Alpha', 'P4_Beta', 'P4_Gamma', 'P4_Slow', 'P4_Low_beta', 'T6_Delta', 'T6_Theta', 'T6_Alpha', 'T6_Beta', 'T6_Gamma', 'T6_Slow', 'T6_Low_beta', 'O1_Delta', 'O1_Theta', 'O1_Alpha', 'O1_Beta', 'O1_Gamma', 'O1_Slow', 'O1_Low_beta', 'O2_Delta', 'O2_Theta', 'O2_Alpha', 'O2_Beta', 'O2_Gamma', 'O2_Slow', 'O2_Low_beta', 'alpha_frontal', 'alpha_temporal', 'alpha_asymmetry', 'beta_frontal', 'beta_temporal', 'beat_asymmetry', 'participant_id', 'label']

In [18]:
len(temp_list)

144

In [19]:
# a = np.array([[2],[4],[6]])
# a.shape

In [20]:
# b = np.array([[1],[2],[3]])

# a/b